In [1]:
# prepare file for ngweight
import csv
texts = list()
project_names = ["jira","stackoverflow","appreviews"]
def read_csv():
    with open('dataset/dataset.csv', 'r',encoding='latin1') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['project'] not in texts:
                texts[row['project']] = list()
                project_names.append(row['project'])
            texts[row['project']].append(row['text'])
            
def write_csv():
    for project_name in project_names:
        with open('dataset/'+project_name+'_n_gram_preparation','w') as file:
            for row in texts[project_name]:
                file.write(chr(0x02)+chr(0x03)+"\n")
                file.write(row+"\n")

read_csv()
print(len(texts))
write_csv()


2767


In [2]:
from subprocess import call
# call("../ngweight-master/waf",shell=True)
for project_name in project_names:
    call("../ngweight-master/bin/default/ngweight -w -s 0 < dataset/"+project_name+"_n_gram_preparation > dataset/"+project_name+"_n_gram",shell=True)

0

In [4]:
import csv
import sys

for project_name in project_names:
    n_gram_filter = list()
    with open("dataset/"+project_name+"_n_gram") as csvfile:
        reader = csv.reader(csvfile, delimiter='\t', quotechar='|')
        for row in reader:
            if int(row[2]) <= 1:
                continue
            n_gram_filter.append(row)

    with open("dataset/"+project_name+"_n_gram_filter",'w') as csvfile:
        writer = csv.writer(csvfile,delimiter='\t', quotechar='|')
        writer.writerows(n_gram_filter)

In [17]:
## check score for each n_gram features
## score = weight1 * int(row[2])
## weight2 = log(|D| * df/sdf^2)
import math

total_document = {"stackoverflow":0,"jira":0,"appreviews":0}

with open("dataset/dataset.csv", 'r',encoding='latin1') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
            total_document[row['project']] += 1

with open('n_gram_filter_summary.csv', 'w', newline='') as writecsvfile:
    fieldnames = ['project_name','|D|','len','gtf','df','sdf','term','weight1','weight1 * gtf','weight2']
    writer = csv.DictWriter(writecsvfile, fieldnames=fieldnames)
    writer.writeheader()
    for project_name in "appreviews","jira","stackoverflow":
        with open("dataset/"+project_name+"_n_gram_filter") as csvfile:
            reader = csv.reader(csvfile, delimiter='\t', quotechar='|')
            for row in reader:
                words = row[5].strip()
                term = tuple(row[5].strip().split(' '))
                weight1 = math.log10(total_document[project_name] / int(row[4]))
                score =  weight1 * int(row[2])
                weight2 = math.log10(total_document[project_name] * int(row[3]) / (int(row[4]) * int(row[4])) )
                writer.writerow({
                    'project_name': project_name,
                    '|D|': total_document[project_name],
                    'len': row[1], 
                    'gtf': row[2],
                    'df': row[3],
                    'sdf': row[4], 
                    'term': row[5],
                    'weight1': weight1,
                    'weight1 * gtf': score,
                    'weight2': weight2,
                })

In [6]:
## count number of n_gram if not use ngweight

import csv

from sklearn.feature_extraction.text import CountVectorizer
corpus = {"jira":[],"stackoverflow":[],"appreviews":[]}

with open("dataset/dataset.csv", 'r',encoding='latin1') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        corpus[row['project']].append(row['text'])

for i in corpus:
    vectorizer = CountVectorizer(ngram_range=(1, 10))
    X = vectorizer.fit_transform(corpus[i])
    print(i,len(vectorizer.get_feature_names()))

jira 37810
stackoverflow 82062
appreviews 62488
